In [1]:
# General stuff
import os
import re
import time
import warnings
import math
import sys
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from subprocess import Popen, PIPE
from os.path import join as opj
from IPython.core.debugger import Tracer
from nilearn.decomposition import CanICA,DictLearning

# sklearn stuff
from sklearn.decomposition import TruncatedSVD,FastICA,SparsePCA,MiniBatchSparsePCA
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import scale
import sklearn.metrics.cluster as metrics
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster.hierarchical import _hc_cut # Internal function to cut ward tree, helps speed up things a lot
from sklearn.utils import resample
from sklearn.model_selection import KFold

# personal functions
from importlib import reload
import fsutils as fs
reload(fs)

if os.path.exists('/data1/vbeliveau/'):
    # On NRU server
    cluster_code='/data1/vbeliveau/5HT_clustering'
    cluster_data='/data1/vbeliveau/5HT_clustering_data'
    subjects_dir='/usr/local/nru/freesurfer/subjects'
elif os.path.exists('C:/Users/vbeliveau/'):
    # On laptop
    cluster_code='C:/Users/vbeliveau/Documents/GitHub/5HT_clustering'
    cluster_data='C:/Users/vbeliveau/Documents/5HT_clustering_data'
    subjects_dir='C:/Users/vbeliveau/Downloads/'
else:
    raise ValueError('Unknown location')
    
# Load local version of nilearn
if os.path.exists('/data1/vbeliveau/'):
    sys.path.append('/data1/vbeliveau/nilearn')
    import mynilearn.decomposition as dcm 
    reload(dcm)
elif os.path.exists('C:/Users/vbeliveau/'):
    sys.path.append('C:/Users/vbeliveau/Downloads/nilearn-master')
    import mynilearn.decomposition as dcm
    reload(dcm)
else:
    raise ValueError('Unknown location')
    
# Analysis directories
surf_data=opj(cluster_data,'surf_data')
fs.assert_dir(surf_data)
BPnd_data=opj(cluster_data,'BPnd')
fs.assert_dir(BPnd_data)
MFA_data=opj(cluster_data,'MFA')
MFA_preproc=opj(MFA_data,'preproc')
fs.assert_dir(MFA_data)
fs.assert_dir(MFA_preproc)

info_path=opj(cluster_code,'info_alltracers_base_healthy_hrrt.mat')
hemi_type=['lh','rh']

info=sio.loadmat(info_path)
tracers=['cumi','dasb','sb','az','C36']
petID=[item for subl in info['petID'] for subsubl in subl for item in subsubl]

In [ ]:
# Cluster BPnd maps using Ward's with spatial constraint
# Dimensionality is estimated based on stability metrics in split-half setting
# This will be useful to see if we gain anything knowledge from decomposing the dynamic data

tracers=['cumi','dasb','sb','az','C36']
# tracers=['cumi']
# hemi_type=['lh','rh']
hemi_type=['lh']
targ='fsaverage5'
sm='10'
    
do_whole=True
do_split=True

K_range=np.arange(2,50,1)
N_iter=10

# Perform Ward clustering on BPnd data
for tracer in tracers:                                            
    for hemi in hemi_type:
        fname=opj(BPnd_data,'maps',tracer,'mrtm2.nopvc.' + targ + '.' + hemi + '.sm' + sm + '.nii.gz')
        fmask=opj(surf_data,'mask.'+targ+'.'+hemi)
        data=fs.load_surf_data(fname,mask=fmask)
        data_scaled=scale(data,axis=0)
        
    dest=opj(BPnd_data,'ward')
    fs.assert_dir(dest)
    nstart=0
    nstop=0
    for hemi in hemi_type:
        adjacency,_=fs.load_surf_neighborhood(opj(surf_data,'neigh.'+targ+'.'+hemi+'.matrix'))
        nstop=nstop+adjacency.shape[0]
        hemi_data=data[nstart:nstop,:]
        if do_whole:        
            fname=opj(dest,tracer + '.'+targ+'.'+hemi+'.nii.gz')
            fmask=opj(surf_data,'mask.'+targ+'.'+hemi)
            fs.ward_clustering(hemi_data,adjacency,mode='whole',K_range=K_range,save_out=fname,
                            mask=fmask,verbose=True)
        if do_split:
            fs.ward_clustering(hemi_data,adjacency,mode='split',K_range=K_range,
                               N_iter=N_iter,verbose=True)    

In [ ]:
# Cluster the 